# 为高性能计算构建容器镜像

本课程将介绍如何使用 [Docker](https://www.docker.com) 和 [Singularity](https://www.sylabs.io/singularity) 构建容器镜像。还将介绍如何使用 [HPC Container Maker](https://github.com/NVIDIA/hpc-container-maker)来简化为高性能计算创建容器规范文件的过程。所涵盖的主题包括容器规范文件、构建容器镜像的基础知识，以及管理容器镜像大小的技巧。

本实验假定您熟悉基本的 Linux shell 命令。

开始之前，请运行以下两个单元，确保正确设置实验环境。要运行某个单元，请突出显示该单元并按下 Ctrl-Enter 或单击工具栏中的“Run”（运行）按钮。

In [ ]:
!docker --version

In [ ]:
!singularity --version

## 为什么 HPC 需要使用容器？

容器是 IT 领域非常常用的技术，但它也适用于高性能计算 (HPC)。

HPC 应用程序通常运行特定于主机。例如，如果在一个系统上构建 HPC 应用程序，然后尝试在另一个 HPC 系统上运行该二进制文件，则可能会是噩梦一般的经历。MPI 和数学库等软件依赖项可能安装在不同的位置，可能是不同的版本，也可能完全缺失。甚至底层 Linux 发行版也可能不同。

容器将整个应用程序用户空间捆绑到一个可移植的软件包中。因此，应用程序环境可移植且保持一致，这与底层系统软件配置无关。容器镜像可被广泛部署，亦可以分享使用，并且无论底层系统如何，结果均能够重现。

容器可以简化系统管理员和最终用户的工作。系统管理员无需维护最终用户请求的数百个相互依赖的软件包。最终用户可以从 [NVIDIA GPU Cloud](https://ngc.nvidia.com)、[Docker Hub](https://hub.docker.com) 或 [Singularity Hub](https://singularity-hub.org) 等存储库下载容器，几分钟内即可运行，无需经历为每个特定系统构建软件的漫长过程。

从容器镜像仓库下载并使用容器镜像是理想选择。但是，如果感兴趣的应用程序环境不可用，该怎么办呢？本课程将介绍如何从头开始构建自己的容器镜像。成功构建应用程序容器镜像后，请考虑将其上传到容器镜像仓库，以便其他人可以从您的工作中获益。

课程结束后，您将为支持 GPU 的真实应用程序构建容器镜像。构建典型 HPC 应用程序代码的应用程序镜像可能需要一些时间。为了届时加快构建速度，我们需要在后台预取一些软件组件。现在不必费心研究它会做什么，但是课程结束时，您应该已经能了解它所做的一切。运行以下单元后，继续下一节。

In [ ]:
%%script bash --bg --out pull.out --err pull.err
docker pull ubuntu:16.04
docker pull centos:7
docker pull nvcr.io/nvidia/nvhpc:20.9-devel-ubuntu20.04
docker pull nvcr.io/nvidia/nvhpc:20.9-runtime-cuda10.1-ubuntu20.04

## 容器镜像格式

[Singularity 容器运行时](https://www.sylabs.io/singularity/)专为高性能计算用例设计。Singularity 容器镜像格式不仅具有无需访问超级用户帐户即可运行容器等特性，还是单个“扁平”文件。这使得 Singularity 容器镜像在系统之间传输并跨集群共享变得更加容易。

那么，为什么本实验还介绍使用 Docker 构建容器镜像？简而言之，Singularity 作为用于 HPC 的容器运行时具有很多优势，而 Docker image builder作为容器镜像构建器具有很多优势。Docker 容器镜像采用[“分层”](https://github.com/opencontainers/image-spec)结构。“分层”镜像的优势包括构建缓存加速构建容器镜像，以及通过多阶段构建可以更精确地控制镜像内容从而将最终容器镜像的大小缩减至最小。

幸运的是，Singularity 可以轻松处理 Docker 镜像。本实验中介绍的关于 HPC 容器的最佳实践如下：

1. 使用 [HPC Container Maker](https://github.com/NVIDIA/hpc-container-maker) 指定容器镜像的内容
2. 使用 Docker 构建容器镜像
3. 将 Docker 镜像转换为 Singularity 镜像
4. 使用 Singularity 在您的 HPC 系统上运行容器

本实验将涵盖这四个主题。

## 使用 Singularity 构建容器镜像

本课程的这一部分将介绍如何[使用 Singularity 构建容器镜像](https://sylabs.io/guides/3.2/user-guide/build_a_container.html)。

构建 Singularity 容器镜像需要管理员权限。与 Docker 不同，运行 Singularity 容器不需要管理员权限。默认情况下，当需要较高权限时，Singularity 使用 `setuid`帮助程序。

### 构建您的第一个 Singularity 镜像：Hello World!

[Singularity 定义文件](https://sylabs.io/guides/3.2/user-guide/definition_files.html)是一个纯文本文件，用于指定创建容器镜像的说明。根据惯例，此文件命名为 `Singularity.def`，但可以使用任何名称。定义文件语法类似于 RPM spec 文件的语法。

对于第一个镜像，我们将使用非常简单的[定义文件](/edit/singularity/Singularity.def)为经典的[“Hello World!”程序](/edit/sources/hello.c)构建容器。Singularity 将基于 Docker Hub 中的 ubuntu:16.04 容器镜像构建新的容器镜像。它会首先尝试在本地查找，然后转至默认容器镜像仓库 (Docker Hub) 下载镜像。

为帮助将镜像大小缩减至最小，Docker Hub 上的 Ubuntu 基础容器不包含开发工具。定义文件需安装 GNU C 编译器和标准 C 头文件。

设置开发环境后，可以从源代码构建“Hello World”程序。

基于定义文件并调用 `singularity build`构建“Hello World”容器镜像。

In [ ]:
!sudo singularity build hello-world.sif singularity/Singularity.def

快速了解 `singularity build`命令行。第一个参数是生成的容器镜像的文件名。根据惯例，Singularity 2.x 容器镜像文件具有 `.simg`扩展名，而 Singularity 3.x 容器镜像具有 `.sif` 扩展名。第二个参数是 Singularity 定义文件的路径。

输出 `Build complete: hello-world.sif`表示镜像已成功构建。

通过调用 `singularity exec`运行容器化的“Hello World”程序。请注意，使用容器镜像时不需要 `sudo`。

In [ ]:
!singularity exec hello-world.sif /usr/local/bin/hello

在容器内运行的 Hello World 程序会生成预期输出。

我们仔细看看 Hello World 容器镜像。

In [ ]:
!ls -lh hello-world.sif

Hello World 程序本身不到 10KB，而 Hello World 容器镜像为 93MB！这是基础 Ubuntu 16.04 镜像大小 (36MB) 的 2.5 倍。编译器占容器总大小的一半以上！我们真正关心的是 Hello World 程序，无需将编译器（或源代码）重新分发给容器镜像的用户。

构建 Hello World 程序之后，您可以通过[删除源代码和编译器](/edit/singularity/Singularity.def.cleanup)缩减 Singularity 容器镜像的大小。如此一来，容器镜像大小可以缩减至 36MB。但是，具有运行时依赖的更复杂的程序将需要更复杂的清理步骤来删除不必要的组件，并维持所需的运行时依赖。

Docker 镜像格式和构建过程包括有助于控制容器镜像大小和更准确地控制容器镜像内容的功能。

### Singularity 总结

Singularity 容器镜像的内容在 Singularity 定义文件中指定。

Singularity 容器镜像是“扁平”的，而不像 Docker (OCI) 镜像一样是分层的。扁平容器镜像是简单的文件，因此易于复制和移动。但是，构建扁平容器镜像无法利用“分层”镜像提供的一些功能特性。

## 使用 Docker 构建容器镜像

本课程的这一部分将介绍如何[使用 Docker 构建容器镜像](https://docs.docker.com/engine/reference/commandline/build/)。

### 构建首个 Docker 镜像

[Dockerfile 是一个纯文本文件](https://docs.docker.com/engine/reference/builder/)，用于指定创建容器镜像的指令。对于第一个镜像，我们将使用非常简单的 [Dockerfile](/edit/docker/Dockerfile.first)。Docker 将基于 Docker Hub 中的 `ubuntu:16.04`容器镜像构建容器镜像。它会首先尝试在本地查找，然后转至默认容器镜像仓库 (Docker Hub) 下载镜像。之后是一条 `RUN`指令，告诉容器构建器（container builder）运行 shell 命令 `date > /build-info.txt` 并将结果保存为容器镜像的一部分。

In [ ]:
!sudo docker build -t first-image -f docker/Dockerfile.first .

快速了解 `docker build`命令行。`-t`选项指定生成的容器镜像的名称和标签，名称和标签由冒号分隔。除非指定了标签，否则默认情况下，Docker 使用 `latest`作为标签。`-f`选项指定用于构建容器的 Dockerfile。最后，`.`是用作构建上下文的路径，即容器镜像构建期间可从主机访问文件的沙盒。

输出 `Successfully tagged first-image:latest`表示镜像已成功构建。

请注意，Dockerfile 的每个指令都显示为“步骤”。在构建容器镜像时，Docker 会告诉您它处于哪个步骤，并给出层的哈希值。

我们来看看新构建的镜像。

In [ ]:
!sudo docker run --rm -it first-image cat /build-info.txt

显示的日期应是不久前构建镜像时的时间。此文件中的日期对应容器镜像的构建时间，而非运行时间。

### 镜像分层

构建容器镜像时，其中一个最重要的概念是*分层*。Docker 根据[开放容器计划 (OCI) 镜像规范](https://github.com/opencontainers/image-spec)构建容器镜像。OCI 容器镜像由一系列层组成。（如果仔细观察构建上面第一个容器镜像的输出，您将看到 `ubuntu:16.04`容器镜像本身实际上由多个层组成。）这些层依次应用，一个在另一个之上，以形成在最终运行容器时看到的容器镜像。

为说明分层，我们[扩展先前的 Dockerfile](/edit/docker/Dockerfile.second)，添加第二条 `RUN` 指令，将构建容器所用系统的 Linux 内核版本附加到 `/build-info.txt`。

In [ ]:
!sudo docker build -t second-image -f docker/Dockerfile.second .

首先，请注意前 2 个步骤已缓存。Docker 可识别前 2 个指令之前已经过处理，因此无需重新生成相应的层。由于分层，可以实现这项功能。层缓存可以显著加快容器镜像的构建速度。回想一下，这些层是按顺序应用的；因此，要使用缓存层，这一步之前指令的整个历史必须完全相同。

刚添加到 Dockerfile 的第三步不在缓存中，因此需要执行此步骤并生成新层。

我们验证一下build-info文件中是否包含内核版本。

In [ ]:
!sudo docker run --rm -it second-image cat /build-info.txt

Docker 提供了一种方法，可以更仔细地查看构成容器镜像的层。

In [ ]:
!sudo docker history second-image

镜像由 7 层组成。这些层按时间倒序列出；运行容器时看到的容器镜像将从显示的最后一层开始生成，将第二层应用于最后一层之上，然后将第三层应用于第二层之上，以此类推。如果发生冲突，后续层将覆盖之前层的内容。

第一列显示层哈希。您可以将此处显示的层哈希与上面的 `docker build`输出相关联。

第二列显示层创建时间。您几分钟前创建了最上面的两层，而其他层对应于 `ubuntu:16.04`基础镜像，是更久之前创建的。

第三列显示用于构建相应层的 Dockerfile 指令的简写版本。要查看完整的指令，请使用 `docker history --no-trunc`。最上面两层的指令与在 [Dockerfile](/edit/docker/Dockerfile.second) 中指定的匹配。

第四列显示层的大小。为什么附加内核版本 (`uname -r ...`) 的层几乎是保存日期的层的两倍大？

OCI 镜像规范使用文件级重复数据来处理冲突。当构建指令创建或修改文件时，整个文件将保存在相应的层中。因此，当内核版本附加到构建信息文件中时，该层不会仅捕获差异，而是捕获整个修改后的文件。因此，文件越小重复数据量极也就越小。考虑1 GB 大型文件的情况。如果后续层修改该文件中的一个字节，即使在运行容器时文件看起来“只有”1GB，但在容器镜像中却要占用 2GB。

对层进行文件级重复数据删除的最佳实践是，将修改同一组文件的所有操作放入同一个 Dockerfile 指令中。例如，在创建临时文件的相同指令中删除所有临时文件。

我们修改一下 Dockerfile，以便[日期和内核版本均在同一指令中写入build-info文件](/edit/docker/Dockerfile.third)。在 bash shell 中，可以用 `&&`将命令连接起来。（您可能已经注意到，在其他 Dockerfile 中很长的 `RUN`命令用 `&&` 连接在一起；这就是原因。）

In [ ]:
!sudo docker build -t third-image -f docker/Dockerfile.third .

In [ ]:
!sudo docker history third-image

请注意，现在构建信息文件只有一层，删除了包含重复数据的无关层。

在使用大量单独的 Dockerfile 指令与使用单一指令之间取得平衡。当涉及相同文件时，大量单独的指令可能会产生不必要的大型容器镜像，但使用过少的指令将消除构建缓存以加快容器构建速度的优势。

最佳实践是将所有*相关*项捆绑放入单个层中，将不相关项放入单独的层中。例如，在一层中安装编译器，在另一层中构建源代码（但清理相同层中的所有临时对象文件）。

###Hello World

我们通过为经典的 [“Hello World!”程序](/edit/sources/hello.c)构建容器镜像，实际练习一下这些技巧。

#### 练习

为帮助将镜像大小缩减至最小，Docker Hub 上的 Ubuntu 基础容器不包含开发工具。作为练习，修改 [Dockerfile](/edit/docker/Dockerfile.hello_exercise) 以安装 GNU C 编译器和标准 C 头文件。对于 Ubuntu，安装软件包的命令是 `apt-get`。软件包命名为 `gcc`和 `build-essential`。

In [ ]:
!sudo docker build -t hello-world:exercise -f docker/Dockerfile.hello_exercise .

通过在容器内运行 Hello World 程序验证您的解决方案。

In [ ]:
!sudo docker run --rm -it hello-world:exercise /usr/local/bin/hello

#### 解决方案

如果遇到问题，或者想比较一下自己的方法，请参阅[解决方案](/edit/docker/Dockerfile.hello_solution)。

请注意，遵循建议的最佳实践，即在创建临时和不必要文件的相同指令中清理这些文件，在生成 apt 包缓存的同一步中删除 apt 包缓存。

In [ ]:
!sudo docker build -t hello-world:solution -f docker/Dockerfile.hello_solution .

在容器内运行的 Hello World 程序会生成预期输出。

In [ ]:
!sudo docker run --rm -it hello-world:solution /usr/local/bin/hello

我们来看看 Hello World 容器镜像中的层。

In [ ]:
!sudo docker history hello-world:solution

Hello World 程序本身不到 10KB，而编译器和相关工具大约 175MB。编译器占容器总大小的*一半以上*！我们真正关心的是 Hello World 程序，无需将编译器（或源代码）重新分发给容器镜像的用户。

### 多阶段 Hello World

Docker [多阶段构建（multi-stage builds）](https://docs.docker.com/develop/develop-images/multistage-build/)是控制容器镜像大小的一种方法。在同一 Dockerfile 中，您可以定义一个完全独立的容器镜像的第二阶段，并且仅将之前阶段中的二进制文件和任何运行时依赖项复制到镜像中。多阶段构建的输出是Dockerfile 的最后一个阶段所对应的一个容器镜像。多阶段 Hello World [Dockerfile](/edit/docker/Dockerfile.hello_multistage) 显示了第二个 `FROM` 指令如何启动第二阶段，但仍然可以访问前一阶段的文件资源 (`COPY --from`)。

In [ ]:
!sudo docker build -t hello-world:multistage -f docker/Dockerfile.hello_multistage .

In [ ]:
!sudo docker history hello-world:multistage

In [ ]:
!sudo docker run --rm -it hello-world:multistage /usr/local/bin/hello

多阶段构建生成的容器镜像仅向基础 `ubuntu:16.04`镜像添加 Hello World 程序，可显著节省镜像大小。多阶段构建还可用于避免重新分发源代码或其他构建构件。但是，请记住这是一种简单的情况，更复杂的情况可能有需要从一个阶段复制到另一个阶段的其他运行时依赖项。HPC Container Maker 有助于确保必要的运行时依赖项在第二阶段可用。

In [ ]:
!sudo docker images hello-world

### Docker 总结

Docker 容器镜像的内容在 Dockerfile 中指定。

Docker (OCI) 容器镜像是分层的。分层提供了许多优势，包括通过缓存来加快构建速度，以及在多个镜像共享层时减少磁盘使用量。但是，分层也需要谨慎使用，以避免出现镜像大小膨胀的陷阱。

多阶段构建是一项非常有用的功能，可用于微调容器镜像的内容。

## HPC Container Maker

[HPC Container Maker (HPCCM)](https://github.com/NVIDIA/hpc-container-maker) 简化了容器规范文件的创建过程。它专门解决了生成 HPC 容器镜像所面临的挑战。

HPC Container Maker 通过 Python recipe 生成 Dockerfile 或 Singularity 定义文件。与“原生”容器规范格式相比，HPCCM recipe 具有一些明显的优势。

1. 一个 HPC 构建块的库，将容器镜像中要包含哪些内容的选择与操作详情分开。构建块以透明方式提供最新的组件和容器最佳实践。

2. 与静态容器规范格式不同，Python 可提供更高的灵活性。基于 Python 的 recipe 可以创建用户输入分支、验证用户输入等。– 相同的 recipe 可以生成多个容器规范。

3. 从相同 recipe 生成 Dockerfile 或 Singularity 定义文件。

### 入门指南

HPCCM 基于[构建块（building blocks）](https://github.com/NVIDIA/hpc-container-maker/blob/master/docs/building_blocks.md)的概念。例如，有一个 [OpenMPI 构建块](https://github.com/NVIDIA/hpc-container-maker/blob/master/docs/building_blocks.md#openmpi)。这些构建块将构建 HPC 软件组件的最佳实践与构建容器镜像的最佳实践封装在一起，以生成最佳容器镜像规范。这样您就可以轻松利用关于如何在容器镜像中以最佳方式安装 OpenMPI 等组件的所有现有知识。

容器镜像被指定为 HPCCM recipe，然后由命令行工具转换为 Dockerfile 或 Singularity 定义文件。HPCCM recipe 是 Python 脚本，通常是非常简单的 Python 脚本。不过，您拥有 Python 的全部功能，因此可以执行验证输入、基于所需构建类型在 recipe 中创建分支，或者搜索 Web 以下载最新版本的软件包等操作。

为了说明这一点，我们先来看一个包含 CUDA 和 OpenMPI 的容器镜像的简单[示例](/edit/hpccm/openmpi.py)。

In [ ]:
!hpccm --recipe hpccm/openmpi.py

当 HPCCM 处理这个简单的两行 recipe 时，会生成经过优化的 Dockerfile。请注意，前面介绍的 Dockerfile 最佳实践（例如将相关步骤合并到单个层，以及在生成临时文件的同一层中删除临时文件）会自动采用。

只需指定 `--format`命令行选项，就可以从完全相同的 recipe 中生成 Singularity 定义文件。

In [ ]:
!hpccm --recipe hpccm/openmpi.py --format singularity

HPCCM 构建块也是可配置的。默认设置适用于许多用例，但您可能需要更精确地定制容器镜像。例如，OpenMPI 构建块具有多个[配置选项](https://github.com/NVIDIA/hpc-container-maker/tree/master/docs/building_blocks.md#openmpi)。

例如，[此 recipe](/edit/hpccm/openmpi-config.py) 将 OpenMPI 安装到 `/opt` 中，禁用 Fortran 接口和 InfiniBand 支持，并指定使用版本 2.1.2。另请注意，基础镜像基于 Ubuntu 而不是 CentOS，就像之前的 recipe 一样；构建块自动检测 Linux 发行版类型，并使用 `apt-get`而不是`yum` 安装其依赖项。

In [ ]:
!hpccm --recipe hpccm/openmpi-config.py

### 重现裸机环境

许多 HPC 系统都使用[环境模块(environment modules)](https://en.wikipedia.org/wiki/Environment_Modules_(software))来管理软件环境。用户加载与所需软件环境对应的模块。

```
$ module load cuda/9.0
$ module load gcc
$ module load openmpi/1.10.7
```

模块可相互依赖，如上例，openmpi 模块由 gcc 编译器构建，并启用 CUDA 支持。

Linux 发行版和驱动通常由系统管理员修复，例如 CentOS 7 和 Mellanox OFED 3.4。

HPC 系统的系统管理员为其用户社区构建并安装了这些组件。并在容器镜像中包含相关软件组件。这需要知道如何正确配置和构建组件。这是专业知识，在应用容器最佳实践时可能会更复杂。

_如何在容器镜像中重现此软件环境？_

任何容器镜像的起点都是基础镜像。由于需要 CUDA，基础镜像应为[公开可用的 CUDA 基础镜像](https://hub.docker.com/r/nvidia/cuda/)中的一个。CUDA 9.0 和 CentOS 7 对应的 CUDA 基础镜像为 `nvidia/cuda:9.0-devel-centos7`。因此，HPCCM recipe 的第一行是：

```python
Stage0 += baseimage(image='nvidia/cuda:9.0-devel-centos7')
```

注意：`Stage0`是指[多阶段 Docker 构建](https://docs.docker.com/develop/develop-images/multistage-build/)的第一个阶段。多阶段构建是一种可以显著缩减容器镜像大小的技巧。本部分不会使用多阶段构建，因此 `Stage0`前缀可视为样板文件。

下一步是包含对应于所需软件环境其余部分的 HPCCM 构建块：[Mellanox OFED](https://github.com/NVIDIA/hpc-container-maker/blob/master/docs/building_blocks.md#mlnx_ofed)、[gcc](https://github.com/NVIDIA/hpc-container-maker/blob/master/docs/building_blocks.md#gnu) 和 [OpenMPI](https://github.com/NVIDIA/hpc-container-maker/blob/master/docs/building_blocks.md#openmpi)。

[mlnx_ofed 构建块](https://github.com/NVIDIA/hpc-container-maker/blob/master/docs/building_blocks.md#mlnx_ofed) 安装 OpenFabrics 用户空间库：

```python
Stage0 += mlnx_ofed(version='3.4-1.0.0.0')
```

[gnu 构建块](https://github.com/NVIDIA/hpc-container-maker/blob/master/docs/building_blocks.md#gnu) 安装 GNU 编译器套件：

```python
compiler = gnu()
Stage0 += compiler
```

注意：通过在此定义`compiler`，以便 OpenMPI 构建块可以在下一步使用 GNU 编译器工具链。由于 GNU 编译器通常是默认编译器，因此这只是对默认行为的显性说明。

[openmpi 构建块](https://github.com/NVIDIA/hpc-container-maker/blob/master/docs/building_blocks.md#openmpi)安装 OpenMPI，配置为使用所需版本的 GNU 编译器，并启用 CUDA 和 InfiniBand：

```python
Stage0 += openmpi(cuda=True, infiniband=True, toolchain=compiler.toolchain,
                  version='1.10.7')
```

综上，与裸机软件环境对应的完整 recipe 是 [cuda-gcc-openmpi.py](/edit/hpccm/cuda-gcc-openmpi.py)。HPCCM recipe 与环境模块命令几乎一一对应。HPCCM 致力于提供类似界面的高级环境模块。只需指定容器镜像中所需的软件组件，而无需深入了解如何以最佳方式构建和配置每个组件。

使用 `hpccm`命令行工具生成相应的 Dockerfile 或 Singularity 定义文件。

In [ ]:
!hpccm --recipe hpccm/cuda-gcc-openmpi.py --format docker

In [ ]:
!hpccm --recipe hpccm/cuda-gcc-openmpi.py --format singularity

根据所需的工作流程，下一步可能是使用文本编辑器将相关步骤添加到构建 HPC 应用程序 Dockerfile 或 Singularity 定义文件中，也可能是扩展 HPCCM recipe 以添加相关步骤构建 HPC 应用程序。

#### 练习

1.修改 [cuda-gcc-openmpi.py](/edit/hpccm/cuda-gcc-openmpi.py) 以使用 GNU 编译器的版本 7。有关详情，请参阅 [gnu 构建块](https://github.com/NVIDIA/hpc-container-maker/blob/master/docs/building_blocks.md#gnu) 文档。

2.修改 [cuda-gcc-openmpi.py](/edit/hpccm/cuda-gcc-openmpi.py) 使用 NVIDA HPC SDK。将 `compiler = gnu()`改为 `compiler = nvhpc(eula=True)`。注意：必须接受 NVIDIA HPC SDK 编译器 EULA 才能使用 [NVIDIA HPC SDK 构建块](https://github.com/NVIDIA/hpc-container-maker/blob/master/docs/building_blocks.md#nvhpc)。

3.修改 [cuda-gcc-openmpi.py](/edit/hpccm/cuda-gcc-openmpi.py)，使 Linux 发行版为 Ubuntu 而不是 CentOS。将基础镜像从 `nvidia/cuda:9.0-devel-centos7`修改为 `nvidia/cuda:9.0-devel-ubuntu16.04`。

#### HPCCM Python 模块

不需要 `hpccm`命令行工具。HPCCM recipe 也可以将 HPCCM Python 模块作为普通 Python 脚本。与前一个 recipe 等同的是 [script-cuda-gcc-openmpi.py](/edit/hpccm/script-cuda-gcc-openmpi.py)。

“recipe”本身完全相同，但 Python 脚本需要额外的代码来导入 Python 模块。解析输入和打印输出是由 hpccm 命令行工具自动处理。不过，脚本也允许精确控制。例如，可以添加其他命令行参数以指定编译器版本、编译器套件、Linux 发行版等。请注意，还可以通过用户参数来定制由 `hpccm` 命令行工具处理的 recipe 的行为。另一个可能的增强功能是将输出写入文件，而不是打印至标准输出。

In [ ]:
!python3 hpccm/script-cuda-gcc-openmpi.py --format docker

### MPI Bandwidth示例程序

来自Lawrence Livermore 国家实验室 (LLNL) 的 [MPI Bandwidth示例程序](/edit/sources/mpi_bandwidth.c) 将用作示例应用程序，说明如何使用 HPCCM recipe 创建应用程序容器。

对于此示例而言，CentOS 7 基础镜像已足够。此外，还需要 Mellanox OFED 用户空间库、编译器和 MPI 库。在教程的这一部分中，我们将使用 GNU 编译器和 OpenMPI。相应的 HPCCM recipe 为：

```python
Stage0 += baseimage(image='centos:7')
Stage0 += gnu(fortran=False)
Stage0 += mlnx_ofed()
Stage0 += openmpi(cuda=False)
```

下一步是从源代码构建 MPI Bandwidth示例程序。首先，必须将源代码复制到容器中，然后进行编译。在这两个步骤中，我们将使用 HPCCM [primitives](https://github.com/NVIDIA/hpc-container-maker/blob/master/docs/primitives.md)。HPCCM primitives是原生容器规范操作的封装器，可将概念操作转换为相应的原生容器特定的语法。primitives还隐藏了 Docker 和 Singularity 容器镜像构建过程之间的许多行为差异，因此无论输出配置规范格式如何，行为均保持一致。

```
Stage0 += copy(src='sources/mpi_bandwidth.c', dest='/var/tmp/mpi_bandwidth.c')
```

注意：MPI Bandwidth示例源代码也可作为容器构建本身的一部分下载，例如使用 wget。与 HPCCM 一起分发的 [MPI Bandwidth示例 recipe](https://github.com/NVIDIA/hpc-container-maker/blob/master/recipes/mpi_bandwidth.py) 就是这样。

最后，使用 mpicc MPI 编译器wrapper编译程序。

```python
Stage0 += shell(commands=[
    'mpicc -o /usr/local/bin/mpi_bandwidth /var/tmp/mpi_bandwidth.c'])
```

注意：在生产容器镜像中，通常还会执行清理步骤以删除源代码和任何其他构建构件。此处跳过了这一步。[多阶段 Docker 构建](https://docs.docker.com/develop/develop-images/multistage-build/)是另一种将应用程序构建过程与应用程序部署分开的方法。

查看完整的 [MPI Bandwidth示例 recipe](/edit/hpccm/mpi_bandwidth.py)。

如果要从容器运行 MPI Bandwidth，请首先生成 Dockerfile。

In [ ]:
!hpccm --recipe hpccm/mpi_bandwidth.py --format docker > Dockerfile.mpi_bandwidth
!cat Dockerfile.mpi_bandwidth

其次，创建 Docker 容器镜像。以下单元将加载 Docker 镜像层的预构建（缓存）版本，直到添加 OpenMPI层，以显著缩短容器镜像构建时间。该操作并非必选项，但如果未加载缓存，MPI Bandwidth容器镜像的构建将需要 10-15 分钟。

In [ ]:
!sudo docker load -i cache/mpi_bandwidth_cache.tar.xz

In [ ]:
!sudo docker build -t mpi_bandwidth -f Dockerfile.mpi_bandwidth .

第三，将 Docker 容器镜像转换为 Singularity 容器镜像。`docker-daemon` endpoint 会告诉 Singularity 使用本地 Docker 镜像库。

In [ ]:
!singularity build mpi_bandwidth.sif docker-daemon://mpi_bandwidth:latest

最后，使用具有 2 个 MPI rank 的 Singularity 运行 MPI Bandwidth示例程序。

In [ ]:
!singularity exec mpi_bandwidth.sif mpirun -n 2 -mca btl_base_warn_component_unused 0 /usr/local/bin/mpi_bandwidth

完全相同的容器镜像也可用于多节点运行，但这不属于本实验的范围。有关多节点 MPI 运行，可参考网络会议[使用 NGC 和 Singularity 扩展 HPC 工作流程](https://info.nvidia.com/simplfying-workflows-with-singularity-reg-page.html?ondemandrgt=yes)。

#### 练习

1.修改 [mpi_bandwidth.py](/edit/hpccm/mpi_bandwidth.py) 以使用 MVAPICH2 而不是 OpenMPI。有关详情，请参阅 [MVAPICH2 构建块](https://github.com/NVIDIA/hpc-container-maker/blob/master/docs/building_blocks.md#mvapich2)。

### 用户参数

使用 Python 表达容器规范是 HPCCM 的一项主要功能。Python recipe 可以处理用户输入，以从相同源代码生成多个容器规范组合。

考虑一下 CUDA 版本和 OpenMPI 版本是用户指定值的情况。如果未指定，应使用默认值。此外，应验证用户提供的值是否为有效的版本号。

hpccm 命令行工具具有 `--userarg`选项。使用此选项指定的值将插入到名为 USERARG 的 Python 字典中，并可以在 recipe 中访问。

它类似于 [`ARG`Dockerfile 指令](https://docs.docker.com/engine/reference/builder/#arg)，但功能更强大，因为您可以使用 Python 处理参数。例如，可以验证输入。

[userargs.py](/edit/hpccm/userargs.py) recipe 展示了用户参数。

In [ ]:
!hpccm --recipe hpccm/userargs.py

In [ ]:
!hpccm --recipe hpccm/userargs.py --userarg cuda=10.0 ompi=3.1.3

#### 练习

1.尝试指定无效或超出范围的用户参数

In [ ]:
!hpccm --recipe hpccm/userargs.py --userarg cuda=nine_point_zero ompi=4.0.0

### 多阶段 Recipe

[多阶段 Docker 构建](https://docs.docker.com/develop/develop-images/multistage-build/)是一项非常实用的功能，可以将应用程序构建步骤与部署步骤分开。在容器内部署已构建的应用程序时，不需要开发工具链、应用程序源代码和编译组件。事实上，它们会显著且不必要地增加容器镜像的大小。

`hpccm`命令行工具可自动创建 2 个阶段：Stage0 和 Stage1。大多数[构建块](https://github.com/NVIDIA/hpc-container-maker/blob/master/docs/building_blocks.md)都提供用于组件相应运行时版本的安装方法。

[multistage.py](/edit/hpccm/multistage.py) recipe 在第一个（构建）阶段安装 GNU 编译器，但在第二个（部署）阶段仅安装相应的运行时库。第一个阶段定义的构建块设置会自动反映在第二个阶段中。

In [ ]:
!hpccm --recipe hpccm/multistage.py

### 多阶段 MPI Bandwidth 示例程序

只需向 recipe 多添加几行，即可将 MPI Bandwidth 示例程序从[单阶段 recipe](/edit/hpccm/mpi_bandwidth.py) 改进为[多阶段 recipe](/edit/hpccm/mpi_bandwidth_multistage.py)。

In [ ]:
!hpccm --recipe hpccm/mpi_bandwidth_multistage.py --format docker > Dockerfile.mpi_bandwidth_multistage
!cat Dockerfile.mpi_bandwidth_multistage

In [ ]:
!sudo docker build -t mpi_bandwidth:multistage -f Dockerfile.mpi_bandwidth_multistage .

In [ ]:
!singularity build mpi_bandwidth_multistage.sif docker-daemon://mpi_bandwidth:multistage

多阶段容器镜像功能相同，但由于开发环境未随 MPI Bandwidth 执行程序进行再分发，因此容器镜像较小。

In [ ]:
!singularity exec mpi_bandwidth_multistage.sif mpirun -n 2 -mca btl_base_warn_component_unused 0 /usr/local/bin/mpi_bandwidth

In [ ]:
!ls -lh mpi_bandwidth*.sif

## miniWeather：一个简单的示例应用程序

[miniWeather 代码](https://github.com/mrnorman/miniWeather)可模拟天气和气候中呈现的基本动力学。动力学本身是由浮力主导的干式可压缩、分层、非静压流动，浮力是静压背景状态下相对较小的扰动。此代码中的方程式本身构成了几乎所有流体动力学代码的主干，它构成了所有天气和气候建模的基础。

它总共包含约 500 行代码（您只关注大约 200 行代码），它通过 MPI + X 提供了并行和移植的方法，其中 X 是 OpenMP、OpenACC、CUDA，或者可能还有其他 CPU 和加速并行方法。

要构建和运行此代码，您需要 MPI、CMake、parallel-netcdf 和 OpenACC 编译器 (NVIDIA HPC SDK)。幸运的是，有适用于所有这些编译器的 HPCCM 构建块。

[HPCCM `nvhpc` 构建块](/edit/hpccm/miniweather_nvhpc_bb.py)可用于编译器，我们将使用 NVIDIA HPC SDK 编译器容器镜像 `nvcr.io/nvidia/nvhpc:20.9-devel-ubuntu20.04`（来自 [NVIDIA GPU Cloud](https://ngc.nvidia.com)）来加快构建过程。NVIDIA HPC SDK 编译器容器镜像是课程开始时预取的镜像。现在下载应该已经完成。

[miniWeather recipe](/edit/hpccm/miniweather.py) 使用 NVIDIA HPC SDK 编译器安装的 OpenMPI。

前两个步骤是从 HPCCM recipe 生成 Dockerfile 并构建相应的 Docker 容器。

构建 miniWeather 容器镜像大约需要 5 分钟，前提是预拉取了 NVIDIA HPC SDK 编译器容器镜像。

In [ ]:
!hpccm --recipe hpccm/miniweather.py > Dockerfile.miniweather
!sudo docker build -t miniweather -f Dockerfile.miniweather .

miniWeather recipe 使用多阶段构建将 Docker 容器镜像大小从大约 9GB 减少至大约 1.5GB。转换为 Singularity 后，最终容器镜像大小为 660MB。[使用 HPCCM `nvhpc` 构建块的配方](/edit/hpccm/miniweather_nvhpc_bb.py) 通过减少运行时库的数量将容器映像大小减少到 120 兆字节以下。

第三，将 Docker 容器镜像转换为 Singularity 容器镜像。这让 Singularity 能够（间接）利用多阶段构建。

In [ ]:
!singularity build miniweather.sif docker-daemon://miniweather:latest

第四也即最后一步是，运行代码的 OpenACC 版本，使用 `--nv`选项在 Singularity 中为单个 MPI rank 启用 GPU 支持。

In [ ]:
!singularity exec --nv miniweather.sif mpirun -n 1 -mca btl_base_warn_component_unused 0 /opt/miniWeather/bin/openacc

代码配置为运行 “injection” 示例。从左侧区域将一股快速微冷的窄气流注入平衡、静止的中性大气中。
这与大气流动无关。只是为了演示。

In [ ]:
import netCDF4
import matplotlib.pyplot as plt
%matplotlib inline
f = netCDF4.Dataset('output.nc', 'r')
theta = f.variables['theta']
plt.imshow(theta[55,:,:], origin='lower')

## 总结

在本实验中，您学习了以下内容：

1. 如何使用 Singularity 构建容器镜像。Singularity 容器镜像是单个“扁平”文件，便于运行时使用。

2. 如何使用 Docker 构建容器镜像。镜像层是一个重要概念，支持缓存构建和多阶段构建。但是，使用不当的镜像层可能会导致不必要的大型容器镜像。

3. HPC Container Maker 是一个开源工具，可以简化容器镜像规范的生存。它可以从 Python recipe 生成 Dockerfile 或 Singularity 定义文件。Python 是一种用于表达式容器规范的更强大的语言，HPCCM 构建块将容器镜像中要包含的 HPC 软件组件的选择与低级别的复杂实现分离。

现在，您应该了解使用此工作流构建 HPC 容器镜像的优势：

1. 使用 [HPC Container Maker](https://github.com/NVIDIA/hpc-container-maker) 指定容器镜像的内容
2. 使用 Docker 构建容器镜像
3. 将 Docker 镜像转换为 Singularity 镜像
4. 使用 Singularity 在您的 HPC 系统上运行容器

## 附录：MILC

MILC 是 MIMD Lattice Computation (MILC) 合作组织编写的一组代码的一部分，这些代码用于研究量子色动力学 (QCD)，即亚原子物理学的强相互作用理论。在 MIMD 并行机器上执行四维 SU(3) 格点规范理论模拟。“强相互作用”负责将夸克结合到质子和中子中，并将它们全部固定在原子核中。

与 miniWeather 或 MPI Bandwidth 不同，MILC 是真实的 HPC 应用程序代码。

HPCCM GitHub 存储库中[包含一个 MILC recipe 示例](https://github.com/NVIDIA/hpc-container-maker/tree/master/recipes/milc)。它证明了多阶段 recipe 的实用性。第一个阶段构建的 Docker 容器镜像为 5.93GB，而采用多阶段构建过程时，容器镜像仅为 429MB。请注意，构建容器镜像需要 30-45 分钟。

In [ ]:
# !curl -O https://raw.githubusercontent.com/NVIDIA/hpc-container-maker/master/recipes/milc/milc.py
!hpccm --recipe hpccm/milc.py > Dockerfile.milc

In [ ]:
!sudo docker build -t milc -f Dockerfile.milc .

In [ ]:
!singularity build milc.sif docker-daemon://milc:latest

就 MILC 而言，从 NVIDIA GPU Cloud (NGC) 使用 MILC 要容易得多，也快得多。与其从头开始构建自己的容器镜像，不如下载 [NGC 中的 MILC 容器](https://ngc.nvidia.com/catalog/containers/hpc:milc)。

In [ ]:
!singularity build milc-ngc.sif docker://nvcr.io/hpc/milc:quda0.8-patch4Oct2017

无论哪种情况，您都可以使用容器镜像在几乎任何系统上轻松运行 MILC。首先，下载样本数据集。

In [ ]:
!mkdir $HOME/milc-dataset
!curl -o $HOME/milc-dataset/benchmarks.tar http://dli-lms.s3.amazonaws.com/data/l-ac-25-v1/benchmarks.tar
!tar -xvf $HOME/milc-dataset/benchmarks.tar -C $HOME/milc-dataset

然后使用 Singularity 运行容器。以下内容配置为使用单个 GPU。完全相同的容器镜像也可用于多节点运行，但这不属于本实验的范围。有关多节点 MPI 运行，可参考网络会议[使用 NGC 和 Singularity 扩展 HPC 工作流程](https://info.nvidia.com/simplfying-workflows-with-singularity-reg-page.html?ondemandrgt=yes)。

第一个单元使用您自己构建的容器镜像，第二个单元使用您从 NGC 下载的容器。

In [ ]:
!singularity exec --nv milc.sif mpirun -n 1 -mca btl_base_warn_component_unused 0 -wdir $HOME/milc-dataset/small su3_rhmd_hisq -geom 1 1 1 1 small.bench.in

In [ ]:
!singularity run --nv milc-ngc.sif --gpu_default_driver=0.0 mpirun -n 1 -mca btl_base_warn_component_unused 0 -wdir $HOME/milc-dataset/small su3_rhmd_hisq -geom 1 1 1 1 small.bench.in

## 附录：使用 Singularity 2.x 转换 Docker 容器镜像

Singularity 3.0 中引入了 `docker-daemon` endpoint。幸运的是，Docker Hub 上有一个方便易用的[容器，可将本地 Docker 镜像转换为 Singularity 2.x 镜像](https://hub.docker.com/r/singularityware/docker2singularity)。

例如，将 MPI Bandwidth容器镜像转换为 Singularty 2.x `simg`：

In [ ]:
!sudo docker run -t --rm --cap-add SYS_ADMIN -v /var/run/docker.sock:/var/run/docker.sock -v /tmp:/output singularityware/docker2singularity mpi_bandwidth

## 附录：术语

- 容器：容器镜像的运行实例化

- 容器镜像：独立软件“包”，包含运行应用程序所需的一切

- 容器运行时：用于运行和管理容器和容器镜像的软件框架。示例：Docker、Singularity。

- 容器镜像仓库：托管用于下载（“拉取”）的容器镜像的服务器。示例：Docker Hub、Singularity Hub、NVIDIA GPU Cloud (NGC)。

## 附录：准备好您的系统容器

在此实验环境中，已为您设置好 Docker 和 Singularity。有关在系统中安装 Singularity 的详情，请参阅此简短[视频](https://dli-lms.s3.amazonaws.com/assets/l-ac-25-v1-zh/NGC_HPC_Singularity_demo.mp4)。